# Import the relevant libraries

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import os
import cv2

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Import the dataset

In [2]:
DIR = "images/datasets/2/"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(DIR, validation_split=0.1, 
                                                                    subset="training", seed=42, 
                                                                    batch_size=16, smart_resize=True)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(DIR, validation_split=0.1,
                                                                  subset="validation", seed=42,
                                                                  batch_size=16, smart_resize=True)

Found 5078 files belonging to 9 classes.
Using 4571 files for training.
Found 5078 files belonging to 9 classes.
Using 507 files for validation.


### Optimise the training and testing performance

In [3]:
classes = train_dataset.class_names
numClasses = len(train_dataset.class_names)
print(classes, numClasses)

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

['Aluminium', 'Carton', 'Glass', 'Organic Waste', 'Other Plastics', 'Paper and Cardboard', 'Plastic', 'Textiles', 'Wood'] 9


### Performing Data Augmentation

In [4]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

preprocess_input = tf.keras.applications.inception_v3.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# MODELS

### Resnet152 Model 

In [5]:
baseModel = tf.keras.applications.ResNet152(input_shape=(256, 256, 3), weights='imagenet',
                                           include_top=False, classes=numClasses)

for layers in baseModel.layers:
    layers.trainable=False
    
last_output = baseModel.layers[-1].output

x = tf.keras.layers.Dropout(0.5)(last_output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(numClasses, activation='softmax')(x)

model = tf.keras.Model(inputs=baseModel.input, outputs=x)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(optimizer=optimizer,
                        loss=loss,
                        metrics=['accuracy'])

epochs=10
callbacks = tf.keras.callbacks.EarlyStopping(patience=2)
history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs, callbacks=[callbacks])

234698864/234698864 [==============================] - 6s 0us/step
Epoch 1/10
286/286 [==============================] - 78s 220ms/step - loss: 0.8630 - accuracy: 0.7587 - val_loss: 0.4245 - val_accuracy: 0.8738
Epoch 2/10
286/286 [==============================] - 55s 190ms/step - loss: 0.1297 - accuracy: 0.9608 - val_loss: 0.4552 - val_accuracy: 0.8619
Epoch 3/10
286/286 [==============================] - 55s 190ms/step - loss: 0.0485 - accuracy: 0.9858 - val_loss: 0.4371 - val_accuracy: 0.8935


### Model Evaluation

In [6]:
# plt.plot(range(0, epochs), history.history["loss"], color="b", label="Loss")
# plt.plot(range(0, epochs), history.history["val_loss"], color="r", label="Test Loss")
# plt.legend()
# plt.show()

In [7]:
# plt.plot(range(0, epochs), history.history["accuracy"], color="b", label="Accuracy")
# plt.plot(range(0, epochs), history.history["val_accuracy"], color="r", label="Test Accuracy")
# plt.legend()
# plt.show()

In [8]:
# plt.xlim([0, 0.003])
# learning_rates = 1e-3 * (10 ** (np.arange(epochs) / 30))
# plt.plot(learning_rates, history.history['loss'], lw=3, color='#48e073')
# plt.title('Learning rate vs. loss', size=20)
# plt.xlabel('Learning rate', size=14)
# plt.ylabel('Loss', size=14)

# Testing

In [9]:
model.save('model.keras')

In [ ]:
# url = "https://images.unsplash.com/photo-1577705998148-6da4f3963bc8?ixid=MnwxMjA3fDB8MHxzZWFyY2h8Nnx8Y2FyZGJvYXJkfGVufDB8fDB8fA%3D%3D&ixlib=rb-1.2.1&auto=format&fit=crop&w=500&q=60"
# image = tf.keras.utils.get_file("images/paper.jpeg", origin=url)

img = tf.keras.preprocessing.image.load_img("images/biodegradable/paper.jpeg", target_size=(256, 256))
img_array = tf.keras.preprocessing.image.img_to_array(img)
print(img_array)
img_array = tf.expand_dims(img_array, 0) 
print("=============")
print(img_array)

predictions = model.predict(img_array)
#score = tf.nn.softmax(predictions[0])

plt.imshow(img)
# print(predictions)
# print("Prediction: " + str(classes[np.argmax(predictions)]))
print(predictions[0]*100, "\n", classes)
print("Prediction: ", classes[np.argmax(predictions)], f"{predictions[0][np.argmax(predictions)]*100}%")

[[[ 21.  16.  20.]
  [ 21.  16.  20.]
  [ 19.  14.  18.]
  ...
  [ 16.  11.  15.]
  [ 18.  13.  17.]
  [ 17.  15.  18.]]

 [[ 19.  14.  20.]
  [ 15.  13.  18.]
  [ 16.  14.  19.]
  ...
  [ 18.  16.  19.]
  [ 17.  15.  18.]
  [ 17.  15.  18.]]

 [[ 17.  15.  18.]
  [ 19.  14.  20.]
  [ 17.  15.  20.]
  ...
  [ 19.  14.  20.]
  [ 17.  15.  18.]
  [ 15.  15.  15.]]

 ...

 [[112. 107. 103.]
  [112. 109. 102.]
  [119. 116. 111.]
  ...
  [ 66.  62.  59.]
  [ 67.  63.  60.]
  [ 64.  60.  59.]]

 [[111. 108. 103.]
  [115. 112. 107.]
  [110. 107. 100.]
  ...
  [ 69.  65.  62.]
  [ 67.  63.  62.]
  [ 67.  63.  60.]]

 [[103. 100.  95.]
  [111. 108. 103.]
  [255. 255. 248.]
  ...
  [ 67.  63.  60.]
  [ 65.  61.  60.]
  [ 62.  58.  55.]]]
tf.Tensor(
[[[[ 21.  16.  20.]
   [ 21.  16.  20.]
   [ 19.  14.  18.]
   ...
   [ 16.  11.  15.]
   [ 18.  13.  17.]
   [ 17.  15.  18.]]

  [[ 19.  14.  20.]
   [ 15.  13.  18.]
   [ 16.  14.  19.]
   ...
   [ 18.  16.  19.]
   [ 17.  15.  18.]
   [ 17.  15.  

In [ ]:
images_ds = tf.keras.utils.image_dataset_from_directory(
    'images',
    labels='inferred',
    label_mode='int',
    batch_size=None
)

def add_batch_dimension(image):
  # Expand the dimension at index 0 (beginning)
  return tf.expand_dims(image, axis=0)

print(images.class_names)
images_ds = images_ds.map(add_batch_dimension)
predictions = model.predict(images_ds)
print(predictions)

In [ ]:
images = tf.keras.utils.image_dataset_from_directory(
    'images',
    labels='inferred',
    label_mode='int',
    batch_size=None
)
print(images.class_names)
for element in images:
    print(element)
    img_array = tf.expand_dims(element, 0)

    predictions = model.predict(img_array)
    #score = tf.nn.softmax(predictions[0])

    plt.imshow(img)
    # print(predictions)
    # print("Prediction: " + str(classes[np.argmax(predictions)]))
    print(predictions[0]*100, "\n", classes)
    print("Prediction: ", classes[np.argmax(predictions)], f"{predictions[0][np.argmax(predictions)]*100}%")